# Using scikit-learn for topic modeling 10 newsgroups

Here we use scikit-learn to topic model a collection of 10 "newsgroups." (Think of a newsgroup as an old-fashioned 90s-era subreddit.)

We use a package called pyLDAvis that makes it possible to explore a model inside a Jupyter notebook.

Then we ask whether KMeans clustering better once the "dimensionality" of a text dataset has been reduced by topic modeling

In [47]:
import re
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler

# parts of this code are adapted from Ben Mabey's sklearn notebook
# here: https://github.com/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# You probably will need to run this before you can run the commands above.
# Uncomment the line below.

# !pip install pyLDAvis

## Fetching the data

Scikit-learn makes it possible for us to download the text of a standard dataset called "20 newsgroups."

Because this is a little large for a class exercise, we'll only download ten of them.

In [2]:
categories10 = ['alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware', 'misc.forsale', 'rec.autos',
               'rec.sport.baseball', 'sci.med', 'sci.space', 'talk.politics.guns', 'talk.religion.misc']

newsgroups10 = fetch_20newsgroups(subset = 'train', categories = categories10, remove=('headers', 'footers', 'quotes'))

#### parts of the data

The newsgroup data comes with the raw texts, in a ```data``` attribute, and labels keyed to the categories we provided, in a ```target``` attribute.

In [3]:
docs_10 = newsgroups10.data

In [4]:
newsgroups10.target[0:10]

array([8, 6, 4, 2, 4, 0, 9, 8, 9, 1])

**let's check that these labels make sense**

The fourth element in that array (index #3 since counting starts at zero) is supposed to be topic ```comp.sys.mac.hardware.```

In [5]:
docs_10[3]

"Hello,\n  I just got some SIMMs,  at least one of which does not work. I don't know if \nthere is a software that can test SIMMs thoroughly or I could just rely on the \nRAM test performed by my computer during the start up. When I installed a dead \nSIMM into an LC or  an LC II, there would be a strange music and no display on \nthe screen. Why? I need your help! Thanks in advance"

Seems right!

## Vectorizing the data

First we need to transform the raw text into a term-document matrix.

In the past we've taken the "sparse matrix" output of the CountVectorizer and turned it into a readable Pandas dataframe, but that's not required here. In fact, the topic modeling function prefers to take a compressed / sparse version directly from the vectorizer.

In [154]:
vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 5)
docterms = vectorizer.fit_transform(docs_10)
print(docterms.shape)

(5528, 9129)


We have 5528 documents, and 9129 different "features" (words).

We're using a regular expression to tokenize the text. If you want to know what it does, you can copy ```\b[a-zA-Z]{3,}\b``` and take it to [https://regex101.com](https://regex101.com) to experiment.

```max_df``` and ```min_df``` select features by document frequency. Floats represent a proportion of documents; integers absolute counts. You can find [full documentation for CountVectorizer at the scikit-learn site.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

## Topic modeling the data 

To produce a topic model, you have to specify the number of topics (```n_components```). 

If you look around online, you will find many different theories and rules of thumb about how to choose the "optimal" number of topics. Many websites will imply that there is some official way to guarantee that you have identified the correct answer. 

But people who are really familiar with topic modeling tend to say that there is no fixed answer to this question. The best number of topics can depend both on your research goal and on the size of the dataset. With very small datasets, it can be appropriate to choose 10 or 20 topics. Very large datasets are sometimes divided into 200 to 400 topics. But the choice may also depend on the *kind* of latent structure you're looking for in the corpus: are you interested in broad patterns or in granular details?

In this case, since we happen to know this dataset is constructed from about 10 different sources, it makes sense to use a number in that range. (Of course, you won't always know anything about the underlying structure. Also, as you'll see, topics are not guaranteed to line up perfectly with the categories a human reader might expect to see.)

In [155]:
lda = LatentDirichletAllocation(n_components=12, random_state=0, max_iter = 15)
lda.fit(docterms)

LatentDirichletAllocation(max_iter=15, n_components=12, random_state=0)

**How do we know whether we've done enough iterations?**

You can check the perplexity. As long as the perplexity keeps going down, the model is probably still getting better. The "perplexity" is calculated by estimating the likelihood of a set of documents given the model; if you don't have a held-out set, you can use the original training set. When the likelihood of the documents goes up, perplexity will go down.

Note, however, that perplexity does not provide a reliable way to compare models or choose the best number of topics. It just helps you judge when the model is converging. And in our classroom work, we're not even going to be very cautious about that, because we don't have time to wait for full convergence.

In [156]:
lda.perplexity(docterms)

3083.2903866769275

In [157]:
pyLDAvis.sklearn.prepare(lda, docterms, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.147056  0.042087       1        1  13.769016
7     -0.013534 -0.186915       2        1  13.273543
9     -0.119193  0.087075       3        1  11.989202
6     -0.196296 -0.030989       4        1  11.171141
4      0.140195 -0.167388       5        1  10.627523
8      0.188306  0.043227       6        1   8.360480
2      0.093619  0.153289       7        1   6.345024
11    -0.140168  0.056805       8        1   5.573594
10    -0.135156 -0.066774       9        1   5.239152
1      0.137460  0.159762      10        1   4.789785
5      0.152356 -0.093302      11        1   4.565801
0      0.039467  0.003121      12        1   4.295738, topic_info=           Term         Freq        Total Category  logprob  loglift
2611        edu  1176.000000  1176.000000  Default  30.0000  30.0000
7646      space  1092.000000  1092.000000  Default  29.0000  29.0000
1511        com   612.000000   612.000000  Default  28.0000  28.0000
3463        god   769.000000   769.000000  Default  27.0000  27.0000
3588        gun   631.000000   631.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
4618       left    44.990222   253.138223  Topic12  -5.8416   1.4201
6978      right    49.808890   738.333552  Topic12  -5.7399   0.4513
3200  following    43.646867   340.682647  Topic12  -5.8720   1.0927
4888        man    43.003366   284.457490  Topic12  -5.8868   1.2582
5936     person    42.352301   331.670825  Topic12  -5.9021   1.0894

[884 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         4  0.918160          aaa
0         8  0.030605          aaa
7         3  0.967120      abiding
33        2  0.972238  accelerator
111       2  0.967421      adapter
...     ...       ...          ...
9112      5  0.049738         york
9112      6  0.021316         york
9112      7  0.042633         york
9112      8  0.390802         york
9112     10  0.468963         york

[2760 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 10, 7, 5, 9, 3, 12, 11, 2, 6, 1])

## Extracting a document-topic matrix

We started out with a matrix that had one column for each term. Now that we've trained a topic model, we can produce a matrix that has one column for each topic.

In [158]:
doctopics = lda.transform(docterms)

In [159]:
doctopics.shape

(5528, 12)

## Try KMeans clustering on this doctopics matrix

From here on, you're mostly on your own. You know enough now to improvise, and this section draws directly on what we did last week.

Do KMeans clustering on *the doc-topic matrix,* (```doctopics```), with n_clusters = 10, to see how well we can reconstruct the divisions of the original dataset. 

There are (imperfect) ways to infer the number of clusters if you don't already know it, and we'll consider them in a moment. But for right now just use n_clusters = 10.

When you get a mapping of documents to clusters (in the ```.labels_``` attribute of the object returned by KMeans), compare it to the original newsgroup assignments using mutual information, to see how close they are.

I'll review mutual information in class, but basically you calculate it by considering the probability of all combinations of two sets of labels. Mutual information will be high if some combinations are much more likely or less likely than we would expect if the labels were independent.

The syntax of this measurement is ```mutual_info_score(one-list-of-labels, the-other-list-of-labels)```. This presumes that the two lists are the same length, and paired.

Then try the same thing doing KMeans clustering on *the document-term matrix* (```docterms```). How many clusters do you get? What's the mutual information with the original, ground-truth newsgroup assignments?


In [44]:
from sklearn.cluster import KMeans
from sklearn.metrics import mutual_info_score

kmeans = KMeans(n_clusters=10, random_state=0).fit(doctopics)

# your code goes here

## Use PCA on doctopics to visualize the documents, colored by KMeans cluster assignment

a few code hints below

In [ ]:
scaler = StandardScaler()

# how would you scale the doctopics matrix?


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
components = pca.fit_transform(scaled_doctopics)  
components.shape

In [1]:
# your code goes here

## Is it possible to infer the correct number of clusters?

In theory, but in practice it doesn't work as well as you might think.

One of the best ways is to compare a) the average distance between points *within* each cluster to b) the average distance *between* clusters. If b is bigger than a, the partitioning is probably strong.

Silhouette Score = (b-a)/max(a,b)

In [ ]:
# This code is based on Om Prakash's answer to this question
# on StackOverflow:
# https://stackoverflow.com/questions/19197715/scikit-learn-k-means-elbow-criterion

from sklearn.metrics import silhouette_score

silhouette_scores = []
n_cluster_vals = []

for n_cluster in range(2, 20):
    kmeans = KMeans(n_clusters=n_cluster, random_state = 0).fit(doctopics)
    label = kmeans.labels_
    sil_coeff = silhouette_score(doctopics, label, metric='euclidean')
    silhouette_scores.append(sil_coeff)
    n_cluster_vals.append(n_cluster)
    
plt.figure()
plt.plot(n_cluster_vals, silhouette_scores)
plt.xlabel("Number of clusters")
plt.ylabel("sil. coefficient")
plt.show() 

max_value = max(silhouette_scores)
print(max_value, n_cluster_vals[silhouette_scores.index(max_value)])

But note that this is more fragile than it appears. In this case, it looks like we're getting near the right value, but this is partly influenced by the number of topics we arbitrarily chose (12).

#### Other things to be aware of

There are different ways to implement Latent Dirichlet Allocation. Most of the implementations that you can load in a Jupyter notebook-- including the scikit-learn implementation--use "variational Bayes" sampling. That has the advantage of being fast. 

But there's pretty robust consensus among people who topic model a lot that "collapsed Gibbs" sampling produces slightly better results. If you're in a situation where you really care about details of the model, it might be worth looking up [a package called MALLET.](http://mallet.cs.umass.edu) This is written in Java and I normally run it from the command line, but there are also Python wrappers with MALLET if you don't want to mess with the command line.